In this script i will generate from the vcf files from slim a table of allele counts per snp for all the populations as an input for baypass 
so the ouput should be, each row a snp positions and each colum the number of population X 2 since the first column is allele ref, the second alt allele

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import math 
import os

import matplotlib.pyplot as plt

import allel
import subprocess

In [2]:
folder_path =  '/Users/tbellagio/tbellagio_cluster/simulations/snakemake/'

In [8]:
## find all the vcf files in the folder 
def find_vcf_paths(folder_path):
    file_pattern = 'subp'
    file_list = []
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('.vcf') and file_pattern in file_name:
                file_path = os.path.join(root, file_name)
                file_list.append(file_path)
    return file_list


In [4]:
### get all the different arquitectures (combinations of pi adn beta)
def substract_arq_folders(folder_list):
    substrings = []
    for string in folder_list:
        start_index = string.find('arq')
        slash_index = string.rfind('/', 0, start_index)
        end_index = string.find('/', start_index)
        substring = string[slash_index + 1: end_index]
        if substring not in substrings:
            substrings.append(substring)

    return substrings

In [5]:
## create a dictionary where the architectures are the keys and the paths including
## and all the vcf files inside those architectures are the data
def create_dict_arq(arqs):
    vcf_arq = {}
    for arq in arqs:
        vcfs_path = [file for file in file_list if arq in file]
        vcf_arq[arq] = vcfs_path
    return vcf_arq

In [30]:
## based on an architecture extract from the dictionary all the vcf files and create 
## two dataframes, one of allele counts where each columns is a ref/alt allele count per optima per subpopulation
## and the other one is a individuals count per population 
def extract_alc_indn_arq(arq):
    allele_counts = pd.DataFrame()
    indiv_number = pd.DataFrame()
    for paths in vcf_arq[arq]:
        ##get name of optima and supb for column name
        start_index = paths.find('optima')
        columns_name = paths[start_index:].replace('/', '_')

        vcf = allel.read_vcf(paths)
        print(len(vcf['variants/POS']))
        genotypes = allel.GenotypeArray(vcf['calldata/GT'])
        ref_allele_counts = genotypes.count_alleles()[:, 0]
        alt_allele_counts = genotypes.count_alleles()[:, 1]

        subp0 = pd.DataFrame(data ={'ref_' + columns_name:ref_allele_counts, 'alt_'+ columns_name: alt_allele_counts}, index = vcf['variants/POS'], )
        allele_counts = pd.concat([allele_counts, subp0], axis=1)
        
        number_of_ind = (subp0.iloc[0, 0] + subp0.iloc[0, 1]) / 2 
        print(number_of_ind)
        indiv_number1 = pd.DataFrame({columns_name:[number_of_ind]})
        indiv_number = pd.concat([indiv_number,indiv_number1],axis=1)


    for col in allele_counts.columns:
        if 'ref_' in col: 
            allele_counts.loc[:,col] = allele_counts.loc[:, col].fillna(indiv_number.loc[0, col[4:]])
        elif 'alt_' in col:
            allele_counts.loc[:,col] = allele_counts.loc[:, col].fillna(0)
    return allele_counts, indiv_number 

In [32]:
## there should be 1536    #852 sun may 14 ##864 sun may 14 ## 1056 Mond may 15 
file_list = find_vcf_paths(folder_path)
len(file_list)

1068

In [16]:
arqs = substract_arq_folders(file_list)

In [17]:
vcf_arq = create_dict_arq(arqs)

In [23]:
number_of_ind = 179.0

In [31]:
for arq in arqs: 
    allele_counts, indiv_number = extract_alc_indn_arq(arq)
    indiv_number.to_csv(f'test/{arq}_indiv_number.csv')
    allele_counts.to_csv(f'test/{arq}_allele_counts.csv')

173571
197.0
173516
117.0
173571
183.0
173577
200.0
173470
151.0
173574
75.0
173562
200.0
173569
200.0
173529
200.0
173558
200.0
173576
113.0
124883
200.0
165165
200.0
165165
200.0
165165
200.0
165165
200.0
165162
200.0
165165
200.0
165165
200.0
165165
200.0
165165
200.0
165165
200.0
128891
200.0
165158
200.0
281822
200.0
273681
200.0
256548
200.0
198869
200.0
228560
200.0
256513
200.0
165031
200.0
256490
200.0
228546
200.0
256530
200.0
165053
200.0
230020
200.0
175270
200.0
175270
200.0
175255
200.0
175270
200.0
204805
200.0
175247
200.0
175270
200.0
175270
200.0
131977
200.0
175266
200.0
175270
200.0
175269
200.0
171835
200.0
171807
200.0
139796
200.0
168914
200.0
197239
200.0
171924
200.0
194405
200.0
139796
200.0
194384
200.0
194326
200.0
199673
200.0
171841
200.0
160740
200.0
85677
200.0
183777
200.0
160739
200.0
173533
200.0
202700
200.0
180562
200.0
180669
200.0
160607
200.0
160721
200.0
125108
200.0
151082
200.0
188102
200.0
188102
200.0
188102
200.0
188080
200.0
188082
200.0
1